Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

I have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like me to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data I provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied me with a csv file called `"bank_marketing.csv"`, which I will need to clean, reformat, and split the data, saving three final csv files. 

In [1]:
import pandas as pd
import numpy as np

# Reading in csv
marketing = pd.read_csv("bank_marketing.csv")

# Splitting into the three tables
client = marketing[["client_id", "age", "job", "marital", 
                    "education", "credit_default", "mortgage"]]
campaign = marketing[["client_id", "number_contacts", "month", "day", 
               "contact_duration", "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics = marketing[["client_id", "cons_price_idx", "euribor_three_months"]]

## Editing the client dataset
# Cleaning education column
client["education"] = client["education"].str.replace(".", "_")
client["education"] = client["education"].replace("unknown", np.NaN)

# Cleaning job column
client["job"] = client["job"].str.replace(".", "_")

# Cleaning and converting client columns to bool data type
for col in ["credit_default", "mortgage"]:
  client[col] = client[col].map({"yes": 1,
                                 "no": 0,
                                 "unknown": 0})
  client[col] = client[col].astype(bool)

# Editing the campaign dataset
# Changing campaign_outcome to binary values
campaign["campaign_outcome"] = campaign["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Converting previous_outcome to binary values
campaign["previous_outcome"] = campaign["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0,
                                                                 "nonexistent": 0})

# Adding year column
campaign["year"] = "2022"

# Converting day to string
campaign["day"] = campaign["day"].astype(str)

# Adding last_contact_date column
campaign["last_contact_date"] = campaign["year"] + "-" + campaign["month"] + "-" + campaign["day"]

# Converting to datetime
campaign["last_contact_date"] = pd.to_datetime(campaign["last_contact_date"], 
                                               format="%Y-%b-%d")

# Cleaning and converting outcome columns to bool
for col in ["campaign_outcome", "previous_outcome"]:
  campaign[col] = campaign[col].astype(bool)

# Dropping unneccessary columns
campaign.drop(columns=["month", "day", "year"], inplace=True)

# Saving tables to individual csv files
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)